ação.
API: https://www.sptrans.com.br/desenvolvedores
ENDPOINTS:https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-r
eferencia/documentacao-api/
Utilizar o GTFS da SPTRANS para dados complementares (Dados
Estáticos/Cadastrais).
Enriquecer os dados de paradas com o endereço da localidade, através de latitude e
longitude.
Dados do GTFS: https://gtfs.org/documentation/schedule/reference/


In [39]:
import requests
import pandas as pd
API_TOKEN = "36b9de7cf0f0bcd8cdde826217615c1b5d2e7b71c80397ec50d1ec7620b4f872"
BASE_URL = "http://api.olhovivo.sptrans.com.br/v2.1"
BASE_PATH = ''

In [27]:
session = requests.Session()
def authenticate(api_token):
     # Define the authentication endpoint
    auth_endpoint = f"{BASE_URL}/Login/Autenticar?token={api_token}"
    
    try:
        # Make a POST request to authenticate
        response = session.post(auth_endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = authenticate(API_TOKEN)
print(result.text)

true


In [28]:
def buscar_linha_sentido(session, termosBusca):
     # Define the authentication endpoint
    endpoint = f"{BASE_URL}/Linha/Buscar?termosBusca={termosBusca}"
    
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False


result = buscar_linha_sentido(session, '8000')
print(result.text)



[{"cl":2506,"lc":false,"lt":"8000","sl":1,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":35274,"lc":false,"lt":"8000","sl":2,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":1273,"lc":false,"lt":"8000","sl":1,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":34041,"lc":false,"lt":"8000","sl":2,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"}]


In [29]:
import json

def generic_get(session, endpoint):
     # Define the authentication endpoint
    endpoint = BASE_URL + endpoint    
    print(endpoint)
    try:
        # Make a POST request to authenticate
        response = session.get(endpoint)
        
        # Check if the authentication was successful
        if response.status_code == 200:
            return response
        else:
            print(f"Erro, status: {response.status_code}")
            return response
    except requests.exceptions.RequestException as e:
        print(f"Erro: {e}")
        return False

def kmz(session):
    return generic_get(session, '/KMZ')

def parada_buscar(session, termos_busca):
    return generic_get(session, f"/Parada/Buscar?termosBusca='{termos_busca}'")

def parada_buscar_por_linha(session, codigo_linha):
    return generic_get(session, f"/Parada/BuscarParadasPorLinha?codigoLinha={codigo_linha}")

def previsao_parada(session, codigo_parada):
    return generic_get(session, f'/Previsao/Parada?codigoParada={codigo_parada}')

def posicao(session):
    return generic_get(session, f'/Posicao')

def posicao_por_linha(session, codigo_linha):
    return generic_get(session, f"/Posicao/Linha?codigoLinha={codigo_linha}")

# paradas = parada_buscar(session, 'rebouças');
# print(json.dumps(result.json(), indent=4))

result = parada_buscar_por_linha(session, '1171')
print(json.dumps(result.json(), indent=4))

# result = previsao_parada(session, '630015011')
# print(json.dumps(result.json(), indent=4))

# result = posicao(session)
# print(json.dumps(result.json(), indent=2))

result = posicao_por_linha(session, 1171)
print(json.dumps(result.json(), indent=2))



http://api.olhovivo.sptrans.com.br/v2.1/Parada/BuscarParadasPorLinha?codigoLinha=1171
[
    {
        "cp": 5614543,
        "np": "JUZA B/C",
        "ed": "ESTRADA DO JUZA",
        "py": -23.822356,
        "px": -46.735702
    },
    {
        "cp": 5614545,
        "np": "EUZ\u00c9BIO COGHI B/C",
        "ed": "RUA EUZ\u00c9BIO COGHI",
        "py": -23.826443,
        "px": -46.732218
    },
    {
        "cp": 5614561,
        "np": "RAVELLO B/C",
        "ed": "RUA TOMAZ SANDRINO",
        "py": -23.824717,
        "px": -46.734245
    },
    {
        "cp": 560009159,
        "np": "JOS\u00c9 NICOLAU DE LIMA B/C",
        "ed": "R JOSE NICOLAU DE LIMA/ AV DO JACEGUAVA",
        "py": -23.777013,
        "px": -46.722914
    },
    {
        "cp": 560009162,
        "np": "ELEVAT\u00d3RIA SABESP B/C",
        "ed": "R NIELS CHRISTIAN SOERENSEN/ R JOSE NICOLAU DE LIMA",
        "py": -23.78024,
        "px": -46.724711
    },
    {
        "cp": 560009165,
        "np": "NIELS C

In [30]:
import openlayer

In [31]:
# !pip install ipyleaflet jupyter-leaflet
# conda install conda-forge::nodejs
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
import ipyleaflet

In [63]:
from ipyleaflet import Map, Marker

center = (-23.5558, -46.6396)

m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=True)
m.add(marker);

In [68]:


result = posicao_por_linha(session, 1171).json()
# print(json.dumps(result.json(), indent=2))
vs = result.get('vs', [])
for veiculo in vs:
    print(veiculo)
    px = veiculo.get('px')
    py = veiculo.get('py')
    print(py,px)
    marker = Marker(location=(py,px), draggable=True)
    m.add(marker);

display(m)

http://api.olhovivo.sptrans.com.br/v2.1/Posicao/Linha?codigoLinha=1171
{'p': '66424', 'a': True, 'ta': '2025-01-21T01:12:13Z', 'py': -23.891852999999998, 'px': -46.730421, 'sv': None, 'is': None}
-23.891852999999998 -46.730421
{'p': '66898', 'a': True, 'ta': '2025-01-21T01:12:36Z', 'py': -23.828041, 'px': -46.731192500000006, 'sv': None, 'is': None}
-23.828041 -46.731192500000006


Map(bottom=2383389.0, center=[-23.832225009179513, -46.721420288085945], controls=(ZoomControl(options=['posit…

In [8]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [42]:
import pandas as pd

BASE_PATH = 'trafi_gtfs'

df_agency = pd.read_csv(f'{BASE_PATH}/agency.txt', header=0)
df_agency.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,1,SPTRANS,http://www.sptrans.com.br/?versao=200125,America/Sao_Paulo,pt


In [43]:
df_calendar = pd.read_csv(f'{BASE_PATH}/calendar.txt', header=0)
df_calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,USD,1,1,1,1,1,1,1,20231001,20250801
1,U__,1,1,1,1,1,0,0,20231001,20250801
2,US_,1,1,1,1,1,1,0,20231001,20250801
3,_SD,0,0,0,0,0,1,1,20231001,20250801
4,__D,0,0,0,0,0,0,1,20231001,20250801


In [44]:
df_fare_attributes = pd.read_csv(f'{BASE_PATH}/fare_attributes.txt', header=0)
df_fare_attributes.head()

,fare_id,price,currency_type,payment_method,transfers,transfer_duration
0,CPTM,5.2,BRL,0,NaN,10800
1,Metrô,5.2,BRL,0,NaN,10800
2,Ônibus,5.0,BRL,0,NaN,10800
3,Ônibus + CPTM,8.9,BRL,0,NaN,10800
4,Ônibus + Metrô,8.9,BRL,0,NaN,10800


In [45]:
df_fare_rules = pd.read_csv(f'{BASE_PATH}/fare_rules.txt', header=0)
df_fare_rules.head()

,fare_id,route_id,origin_id,destination_id,contains_id
0,CPTM,CPTM L07,NaN,NaN,NaN
1,CPTM,CPTM L08,NaN,NaN,NaN
2,CPTM,CPTM L09,NaN,NaN,NaN
3,CPTM,CPTM L10,NaN,NaN,NaN
4,CPTM,CPTM L11,NaN,NaN,NaN


In [46]:
df_frequencies = pd.read_csv(f'{BASE_PATH}/frequencies.txt', header=0)
df_frequencies.head()

,trip_id,start_time,end_time,headway_secs
0,1012-10-0,00:00:00,00:59:00,1200
1,1012-10-0,01:00:00,01:59:00,3600
2,1012-10-0,05:00:00,05:59:00,600
3,1012-10-0,06:00:00,06:59:00,600
4,1012-10-0,07:00:00,07:59:00,600


In [47]:
df_routes = pd.read_csv(f'{BASE_PATH}/routes.txt', header=0)
df_routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF
1,1012-21,1,1012-21,Term. Jd. Britânia - Jd. Rosinha,3,509E2F,FFFFFF
2,1014-10,1,1014-10,Morro Doce - Cptm Perus,3,509E2F,FFFFFF
3,1015-10,1,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,3,509E2F,FFFFFF
4,1016-10,1,1016-10,Cem. Do Horto - Shop. Center Norte,3,002F6C,FFFFFF


In [55]:
df_shapes = pd.read_csv(f'{BASE_PATH}/shapes.txt', header=0)
shape_pos = df_shapes[df_shapes['shape_id'] == 81072][['shape_pt_lat', 'shape_pt_lon']].values 

In [75]:
from ipyleaflet import Map, Marker, FullScreenControl

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

Map(center=[-23.436078552542355, -46.79888413220338], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [49]:
df_stop_times = pd.read_csv(f'{BASE_PATH}/stop_times.txt', header=0)
df_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1012-10-0,18:00:00,18:00:00,301790,1
1,1012-10-0,18:01:35,18:01:35,301764,2
2,1012-10-0,18:03:10,18:03:10,301724,3
3,1012-10-0,18:04:45,18:04:45,301730,4
4,1012-10-0,18:06:20,18:06:20,30003042,5


In [77]:
df_stops = pd.read_csv(f'{BASE_PATH}/stops.txt', header=0)
print(df_stops.shape)
df_stops.head()


(21842, 5)


,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,18848,Clínicas,NaN,-23.554022,-46.671108
1,18849,Vila Madalena,NaN,-23.546498,-46.691141
2,18850,Consolação,NaN,-23.558094,-46.660205
3,18851,Conceição,NaN,-23.635039,-46.641239
4,18852,Jabaquara,NaN,-23.646033,-46.641028


In [78]:
from ipyleaflet import Map, Marker, FullScreenControl
shape_pos = df_stops[['stop_lat', 'stop_lon']].values[:500] 

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

Map(center=[-23.524372034, -46.63580856000003], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [79]:
df_trips = pd.read_csv(f'{BASE_PATH}/trips.txt', header=0)
df_trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073
2,1012-21,U__,1012-21-0,Jd. Rosinha,0,81195
3,1014-10,U__,1014-10-0,Cptm Perus,0,82855
4,1015-10,USD,1015-10-0,Chác. Maria Trindade,0,81148
